In [ ]:
import numpy as np
import ipywidgets as widgets
from ipywidgets import VBox, HBox
import pythreejs as three
import unray as ur

def setup_renderer(group, *,
                   scale=1.0, camera_direction=(1, 1, 1), light_direction=(0, 1, 1),
                   width=800, height=600, background='#eeeeee'):
    "Helper function to setup a basic pythreejs renderer and scene, adding given group to it."
    camera_position = tuple(map(lambda x: x*scale, camera_direction))
    light_position = tuple(map(lambda x: x*scale, light_direction))
    camera = three.PerspectiveCamera(
        position=camera_position,
        aspect=width/height
    )
    key_light = three.DirectionalLight(position=light_position)
    ambient = three.AmbientLight(intensity=0.5)
    scene = three.Scene(children=[key_light, ambient, camera, group], background=background)
    controls = three.OrbitControls(camera)
    renderer = three.Renderer(scene, camera, [controls],
                              width=width, height=height)
    return renderer

def display_plots(*plots, **kwargs):
    "Display all plots in a single renderer. Returns renderer."
    group = three.Group()
    for plot in plots:
        group.add(plot)
    renderer = setup_renderer(group, **kwargs)
    return renderer

def single_tetrahedron():
    cells = np.zeros((1, 4), dtype="int32")
    coordinates = np.zeros((4, 3), dtype="float32")
    cells[0, :] = [0, 1, 2, 3]
    coordinates[0, :] = [0, 0, 0]
    coordinates[1, :] = [1, 0, 0]
    coordinates[2, :] = [0, 1, 0]
    coordinates[3, :] = [0, 0, 1]
    values = np.zeros(4, dtype="float32")
    values[:] = [1, 3, 2, -1]
    return cells, coordinates, values

cells_array, points_array, function_array = single_tetrahedron()
mesh = ur.Mesh(cells=cells_array, points=points_array)
field = ur.Field(mesh=mesh, values=function_array)
scale = max(*[points_array[:,i].max() - points_array[:,i].min() for i in (0,1,2)])
fmin = field.values.min()
fmax = field.values.max()
favg = 0.5 * (fmin + fmax)
fdiff = fmax - fmin
lut = ur.ArrayColorLUT(values=[[1,0,0], [0,1,1]])
color = ur.ColorField(field=field, lut=lut)

In [ ]:
params = ur.IsovalueParams(value=favg, spacing=0.1*fdiff, mode="linear")
plot = ur.IsosurfacePlot(mesh=mesh, color=color, values=params)

In [ ]:
renderer = display_plots(plot, scale=scale)
renderer

In [ ]:
db = plot.dashboard()
db

In [ ]:
slider = widgets.FloatSlider(value=favg, min=fmin, max=fmax, description="Value")
widgets.jslink((slider, "value"), (params, "value"))
slider